In [96]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

import pandas as pd
from sklearn.manifold import TSNE
from ast import literal_eval

df = pd.read_csv("./cds-stellantis-symptoms-lookup-table_export.csv")
df = df[["symptoms", "symp_embeddings"]]
df['cat'] = 0 
df['cat'] = np.where(df['symptoms'].isin(["cel on", "check engine light on", "engine light on", "Check Engine Light"]), 1, 0)
matrix = np.array(df['symp_embeddings'].apply(literal_eval).to_list())

df_cat0 = df[df['cat'] == 0]#.sample(150)
df_cat1 = df[df['cat'] == 1]
df = pd.concat([df_cat0, df_cat1])

In [97]:
df

,symptoms,symp_embeddings,cat
0,Aftertreatment Diesel Exhaust Fluid Consumptio...,"[-0.008613779, 0.010551015, -0.0058428408, -0....",0
1,Aftertreatment Engine Out NOx Sensor Test Did ...,"[-0.0137487175, 0.01049209, -0.015019271, 0.00...",0
2,Aftertreatment Selective Catalyst Reduction Sy...,"[-0.025860835, 9.3502214E-4, 0.0064443536, 0.0...",0
3,Air Handling Performance Test did not complete,"[-0.030768884, 0.005781261, -0.0089293355, -0....",0
4,Coolant Temperature High,"[4.7822535E-4, 0.012556653, -7.3425344E-4, -0....",0
...,...,...,...
851,wont start,"[-0.027165346, -0.011356885, 0.0011455772, 1.6...",0
41,check engine light on,"[-0.019812554, -0.008862461, 0.012000557, -0.0...",1
58,engine light on,"[-0.006408318, -0.00855313, 0.014625199, -0.03...",1
337,Check Engine Light,"[-0.01904354, 0.009402665, 0.01181123, -0.0204...",1


In [98]:
# For 2D


matrix = np.array(df['symp_embeddings'].apply(literal_eval).to_list())
# Create a t-SNE model and transform the data
tsne = TSNE(n_components=2, perplexity=15, random_state=42, init='random', learning_rate='auto')
vis_dims = tsne.fit_transform(matrix)
vis_dims.shape
colors = ["red", "blue"]
x = [x for x, y in vis_dims]
y = [y for x, y in vis_dims]
cat = df['cat'].tolist()
symp = df['symptoms'].tolist()

# pd.DataFrame(zip(x,y))


# fig = plt.figure()
# ax = fig.add_subplot()
# for i in range(0, len(x)):
#     if cat[i] == 1:
#         ax.text(x[i], y[i], symp[i])
#         plt.scatter(x[i], y[i], color = 'red')
#     else:
#         plt.scatter(x[i], y[i], alpha = 0.3, color = "blue")
# plt.show()



In [99]:
# len(z)


In [103]:
# For 3D
# Create a t-SNE model and transform the data
import plotly.io as pio
pio.renderers.default = "browser"
tsne = TSNE(n_components=3, perplexity=15, random_state=42, init='random', learning_rate='auto')
vis_dims = tsne.fit_transform(matrix)
x = [x for x, y, z in vis_dims]
y = [y for x, y, z in vis_dims]
z = [z for x, y, z in vis_dims]

cat = df['cat'].tolist()
symp = df['symptoms'].tolist()

df_3d_cat0 = pd.DataFrame(zip(x,y,z, cat, symp), columns = ['x', 'y', 'z', 'cat', 'symptom'])[:len(x)-4]

df_3d_cat1 = pd.DataFrame(zip(x,y,z, cat, symp), columns = ['x', 'y', 'z', 'cat', 'symptom'])[-4::]
#df_3d
plot1 = px.scatter_3d(df_3d_cat0, x='x', y='y', z='z', color_discrete_sequence= ['blue'], opacity = 0.3, size_max = 3)
plot2 = px.scatter_3d(df_3d_cat1, x='x', y='y', z='z', color_discrete_sequence = ['red'])

fig = go.Figure(data= plot1.data+plot2.data)

label_x = df_3d_cat1['x'].tolist()
label_y = df_3d_cat1['y'].tolist()
label_z = df_3d_cat1['z'].tolist()
label_cat = df_3d_cat1['cat'].tolist()
label_symptom = df_3d_cat1['symptom'].tolist()

to_update = []
for x_in,y_in,z_in,cat_in, symp_in in zip(label_x, label_y, label_z, label_cat, label_symptom):
    #fig.add_annotation(x=x, y=y, z=z, text = symp, showarrow=True,arrowhead=1)
    to_update.append(dict(showarrow = True, x = x_in, y = y_in, z = z_in, text = symp_in,arrowhead = 5))

fig.update_layout(scene = dict(annotations= to_update))
# fig.update_layout(scene = dict(annotations=[dict(showarrow = True, x = 0, y = 0, z = 0, text = "TESTING")]))
fig.show()


In [105]:
from sklearn.cluster import KMeans
# https://medium.com/@sk.shravan00/k-means-for-3-variables-260d20849730#id_token=eyJhbGciOiJSUzI1NiIsImtpZCI6ImUyNmQ5MTdiMWZlOGRlMTMzODJhYTdjYzlhMWQ2ZTkzMjYyZjMzZTIiLCJ0eXAiOiJKV1QifQ.eyJpc3MiOiJodHRwczovL2FjY291bnRzLmdvb2dsZS5jb20iLCJhenAiOiIyMTYyOTYwMzU4MzQtazFrNnFlMDYwczJ0cDJhMmphbTRsamRjbXMwMHN0dGcuYXBwcy5nb29nbGV1c2VyY29udGVudC5jb20iLCJhdWQiOiIyMTYyOTYwMzU4MzQtazFrNnFlMDYwczJ0cDJhMmphbTRsamRjbXMwMHN0dGcuYXBwcy5nb29nbGV1c2VyY29udGVudC5jb20iLCJzdWIiOiIxMDA4MzU2ODYwMjU4MDU5OTIzNzciLCJlbWFpbCI6Imt0aWlhbm5AZ21haWwuY29tIiwiZW1haWxfdmVyaWZpZWQiOnRydWUsIm5iZiI6MTcyMjY0MDE4MiwibmFtZSI6IktldmluIFRpYW4iLCJwaWN0dXJlIjoiaHR0cHM6Ly9saDMuZ29vZ2xldXNlcmNvbnRlbnQuY29tL2EvQUNnOG9jSVFkQWRXdk1uR0N1RE1uV1VqN05MSU5Ld3hPbkV4UUx3WkhwelRXQnpkR1VsV1FBPXM5Ni1jIiwiZ2l2ZW5fbmFtZSI6IktldmluIiwiZmFtaWx5X25hbWUiOiJUaWFuIiwiaWF0IjoxNzIyNjQwNDgyLCJleHAiOjE3MjI2NDQwODIsImp0aSI6IjU1NjM3M2EzMzFiOTE1MzA0YjViMjI1MTE5NzVjMDA3ZTM0YzQ4N2IifQ.l-dZtKsULbTfN4_GS7CSQG5fAmod2CqIJpTJ42xewJ3XlehiuVc9baeEoQRb_iqpNxQ-3Pfw8FmLoC5BVD9CoV-5f0prFI17ZQFW-DiXka1GLozJZNdzq2Js-Qd12We-WlBiCiU6zKkjEUs1AR946JVo2Poinz8MPqE8Er9Hi41hNsFXfb0wYNIWqjVFdfAWdkMeEc0te5vWpT-nKYkrBdelvfGKtLLK4yHjuIXDe0y73iJV2IzFv3MDMUXcK6McqtPjZlqWEt7a4H-CAXgNgTnnmn0lImP8Rqtv64qWe24Yd-PQRgDVtzCsivv8WhQMi3zwKyBKfL8OOx5rUUFFKA


AttributeError: 'Series' object has no attribute '_validate_params'